# Getting Started
Using Conversational React Agent to interact with our llm tools

In [1]:
from togetherllm import TogetherLLM
from langchain.tools import DuckDuckGoSearchRun

from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.agents import load_tools
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain.agents import AgentType

from pprint import pprint

from langchain.chains.summarize import load_summarize_chain
from langchain.chains import LLMMathChain

from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate, LLMChain

In [2]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature=0,
    max_tokens=512
)

In [3]:
tools = []

In [4]:
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

In [5]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [6]:
print(llm_chain.run("write me a python code that convert excel to csv"))



import pandas as pd

# Read Excel file
df = pd.read_excel('file.xlsx')

# Convert to CSV
df.to_csv('output.csv', index=False)

# Write the CSV file
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Name', 'Age', 'Gender'])
    for row in df:
        writer.writerow(row)

This code reads an Excel file using pandas and converts it to a CSV file. The `to_csv()` method is used to write the data to a CSV file. The `index=False` parameter is used to exclude the sheet index from the CSV file. The `with open()` statement is used to open the CSV file in write mode and create a csv writer object. The `writerow()` method is used to write each row of the data to the CSV file.

You can also use the `to_csv()` method with the `index=False` parameter to write the data to a CSV file without the sheet index.
```
df.to_csv('output.csv', index=False)
```
You can also use the `to_csv()` method with the `orient='index'` parameter to write the data

## Let's create our Tool
- 1st Tool : To create summarizer tools
- 2nd Tool : To create Q&A tools
- 3rd Tool : To create LLM Math Chain tools
- 4th Tool : To create a general tools

In [ ]:
### summarize
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")
summarize_general = Tool(
    name='summarization Tool',
    func=summarize_chain.run,
    description='use this tool when summarizing information'
)

In [ ]:
## QA closed book

custom_prompt_template = """[INST] <<SYS>>
Your name is Kelly, You are helpful assistant, you always open and only answer for the assistant then you stop.
If you don't know the answer, just say you don't know and submit the request to hotline@xfab.com for further assistance.
<</SYS>>

Question: {question}
[/INST]"""

#prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

prompt = PromptTemplate(input_variables=['question'],template=custom_prompt_template)
qachain = LLMChain(llm=llm, prompt=prompt)
QA_general = Tool(
    name='Question Answer',
    func=qachain.run,
    description='use this tool when answering questions'
)

In [7]:
### Math 
llm_math = LLMMathChain(llm=llm, verbose=True)
llm_math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='use this tool to perform calculations'
)

/home/jlukas/Desktop/My_Project/AI_CTS/aicts/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:51: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
### General Tools
llm_chain = LLMChain(llm=llm, prompt=prompt)
genneral_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

## Here we combine the Tool 

In [ ]:
tools.extend([summarize_general, QA_general, llm_math_tool, genneral_tool])

## Here we'll initialize the agent 

In [ ]:
# Using Zero Shot
agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    #verbose=True,
    max_iterations=3,
    handle_parsing_errors="Check your output and make sure it conforms!",
    early_stopping_method="generate", #to perform one FINAL pass through the LLM to generate an output
)

In [ ]:
# Using conversational react with memory
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=4,
    memory=memory,
    handle_parsing_errors="Check your output and make sure it conforms!",
    early_stopping_method="generate", #to perform one FINAL pass through the LLM to generate an output
)

In [ ]:
agent("Write me python code that convert excel to csv ?")

In [ ]:
try:
    response= agent.run("Write me a python code that convert excel to csv?")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [ ]:
try:
    response= agent.run("Please Write me an arduino code that read the ultrasonic sensor distance and turn the buzzer if distance less than 5cm")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [ ]:
try:
    response= agent.run("Help me to find x to equation , find x for 4x+2 = 5x")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [ ]:
try:
    response= agent.run("Show me the calculation step")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")